In [ ]:
import ccxt
import pandas as pd
import time  # Для задержек, если лимиты API

def fetch_ohlcv(symbol, timeframe, since, limit=1000):
    """
    Функция для загрузки OHLCV-данных с Binance через CCXT.
    
    :param symbol: Символ токена, например 'BTC/USDT'
    :param timeframe: Таймфрейм, например '1m', '5m', '1h', '1d'
    :param since: Начальная дата в формате ISO8601, например '2025-05-01T00:00:00Z'
    :param limit: Максимальное число свечей на запрос (по умолчанию 1000)
    :return: DataFrame с колонками: timestamp, open, high, low, close, volume
    """
    exchange = ccxt.binance()  # Можно заменить на ccxt.kucoin() или другую
    since_ms = exchange.parse8601(since)  # Конвертируем в миллисекунды
    
    all_ohlcv = []  # Список для всех свечей
    while True:
        try:
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since_ms, limit)
            if not ohlcv:
                break  # Нет больше данных
            all_ohlcv.extend(ohlcv)
            since_ms = ohlcv[-1][0] + 1  # Следующий запрос с последней свечи +1 мс
            print(f"Загружено {len(ohlcv)} свечей, продолжаем...")
            time.sleep(1)  # Задержка для избежания rate-limit
        except ccxt.NetworkError as e:
            print(f"Ошибка сети: {e}. Повтор через 5 сек...")
            time.sleep(5)
        except ccxt.ExchangeError as e:
            print(f"Ошибка биржи: {e}")
            break
    
    if all_ohlcv:
        df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')  # Конвертируем в читаемую дату
        return df
    else:
        print("Нет данных для этого периода.")
        return pd.DataFrame()

# Пример использования
symbol = 'DOGE/USDT'#'BNB/USDT'#'ETH/USDT'#'SOL/USDT'#'BTC/USDT'
timeframes = ['1d', '1h', '1m']  # Разные таймфреймы для теста
since = '2025-05-01T00:00:00Z'  # Начало периода (май 2025)

for tf in timeframes:
    df = fetch_ohlcv(symbol, tf, since)
    if not df.empty:
        print(f"\nДанные для {tf}:")
        print(df.tail(5))  # Последние 5 строк
        df.to_csv(f'doge_usdt_{tf}.csv', index=False)  # Сохраняем в CSV

In [8]:
from dune_client.client import DuneClient
dune = DuneClient("FRj6Bysj3Cs0Uy0WTihbWG43oDLu2CLH")
query_result = dune.get_latest_result(41840)

In [24]:
import pandas as pd

# Extract the list of rows from the ResultsResponse
rows = query_result.get_rows()

# Create the DataFrame
df = pd.DataFrame(rows)
df

,# Receiving Addresses (R),# Sending Addresses (S),# Total Addresses,Baseline,Day,S/R Ratio
0,273018,543235,626034,1,2025-08-04 00:00:00.000 UTC,1.99
1,243543,483382,561709,1,2025-08-03 00:00:00.000 UTC,1.98
2,278274,529392,610919,1,2025-08-02 00:00:00.000 UTC,1.90
3,300429,572052,667830,1,2025-08-01 00:00:00.000 UTC,1.90
4,365144,697324,804438,1,2025-07-31 00:00:00.000 UTC,1.91
...,...,...,...,...,...,...
1995,138660,205602,259781,1,2020-02-17 00:00:00.000 UTC,1.48
1996,129013,192740,243738,1,2020-02-16 00:00:00.000 UTC,1.49
1997,142740,204520,260385,1,2020-02-15 00:00:00.000 UTC,1.43
1998,150693,213564,271864,1,2020-02-14 00:00:00.000 UTC,1.42


In [25]:
df.to_csv('out.csv')  

# DEX + Noname Coins

In [6]:
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
import json
import os

# Function to get OHLCV data for a pool, paginating backward from to_timestamp
def get_ohlcv(network, pool_address, from_timestamp, to_timestamp, filename, timeframe='minute', aggregate=1, currency='usd', token='base'):
    url = f"https://api.geckoterminal.com/api/v2/networks/{network}/pools/{pool_address}/ohlcv/{timeframe}"
    headers = {'Accept': 'application/json'}
    
    # Load existing data
    ohlcv_all = []
    last_fetched_ts = to_timestamp  # Start from the end
    if os.path.exists(filename):
        df_existing = pd.read_csv(filename)
        ohlcv_all = df_existing.values.tolist()
        if ohlcv_all:
            ohlcv_all.sort(key=lambda x: x[0])  # Ascending order
            last_fetched_ts = int(ohlcv_all[0][0])  # Resume from oldest timestamp
            print(f"Resuming from oldest timestamp: {datetime.fromtimestamp(last_fetched_ts)}")
    
    current_before = last_fetched_ts
    
    while True:
        params = {
            'aggregate': aggregate,
            'before_timestamp': current_before,
            'limit': 1000,
            'currency': currency,
            'token': token
        }
        try:
            print(f"Requesting OHLCV: {url} with before_timestamp={datetime.fromtimestamp(current_before)}")
            response = requests.get(url, params=params, headers=headers)
            response.raise_for_status()
            data = response.json()
            ohlcv_list = data['data']['attributes']['ohlcv_list']
            
            if not ohlcv_list:
                print(f"No OHLCV data returned for {network}/{pool_address} before {datetime.fromtimestamp(current_before)}. Likely reached pool creation.")
                break
            
            # Sort ascending and filter to avoid duplicates
            ohlcv_list.sort(key=lambda x: x[0])
            new_ohlcv = [candle for candle in ohlcv_list if from_timestamp <= candle[0] < current_before]
            
            if not new_ohlcv:
                print(f"No new OHLCV data in range for {network}/{pool_address}")
                break
            
            # Prepend new data (since fetching older data)
            ohlcv_all = new_ohlcv + ohlcv_all
            save_ohlcv_to_csv(ohlcv_all, filename)
            
            oldest_ts = new_ohlcv[0][0]
            print(f"Fetched {len(new_ohlcv)} candles, oldest: {datetime.fromtimestamp(oldest_ts)}")
            if oldest_ts <= from_timestamp:
                print(f"Reached start of range: {datetime.fromtimestamp(from_timestamp)}")
                break
            current_before = oldest_ts
            
            time.sleep(2)
        except requests.exceptions.HTTPError as e:
            print(f"HTTP error for {network}/{pool_address}: {e}, status code: {e.response.status_code}")
            if e.response.status_code == 429:
                print("Rate limit exceeded. Waiting 60 seconds...")
                time.sleep(60)
                continue
            elif e.response.status_code == 404:
                print(f"Pool {pool_address} not found. Marking as failed.")
                save_ohlcv_to_csv(ohlcv_all, filename)
                return ohlcv_all
            elif e.response.status_code == 422:
                print(f"Pool {pool_address} has >2 tokens, not supported. Marking as failed.")
                save_ohlcv_to_csv(ohlcv_all, filename)
                return ohlcv_all
            save_ohlcv_to_csv(ohlcv_all, filename)
            break
        except Exception as e:
            print(f"General error for {network}/{pool_address}: {e}")
            save_ohlcv_to_csv(ohlcv_all, filename)
            break
    
    ohlcv_filtered = [candle for candle in ohlcv_all if from_timestamp <= candle[0] <= to_timestamp]
    ohlcv_filtered.sort(key=lambda x: x[0])
    return ohlcv_filtered

# Helper to save OHLCV to CSV
def save_ohlcv_to_csv(ohlcv_data, filename):
    if not ohlcv_data:
        print(f"No data to save for {filename}")
        return
    df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume_usd'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df.to_csv(filename, index=False)
    print(f"Saved to {filename}. Rows: {len(df)}")

# Save/load progress
def save_progress(progress_file, key, status, last_ts=None):
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            progress = json.load(f)
    else:
        progress = {}
    
    progress[str(key)] = {'status': status, 'last_ts': last_ts}
    
    with open(progress_file, 'w') as f:
        json.dump(progress, f)
    print(f"Updated progress: {key} -> {status}")

def load_progress(progress_file):
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            return json.load(f)
    return {}

In [ ]:
# Main script
pools = [
    {'network': 'solana', 'pool_address': 'EXQfMJKTnhHRv3xCzPttk9sZp6MKMdipeDnjKbGJq4fy', 'from_date': '2025-05-01', 'to_date': '2025-08-25'},
    {'network': 'solana', 'pool_address': 'EwUU8oiKTomjekU5eszctx3kneKJnKm5BJpn5sTT4CUf', 'from_date': '2025-05-01', 'to_date': '2025-08-25'},
    {'network': 'solana', 'pool_address': 'BgdLpSBcEny1QZLMQGxW42az4iAb38ijwq5bxyHTs1mV', 'from_date': '2025-05-01', 'to_date': '2025-08-25'},
    {'network': 'solana', 'pool_address': '7rQd8FhC1rimV3v9edCRZ6RNFsJN1puXM9UmjaURJRNj', 'from_date': '2025-05-01', 'to_date': '2025-08-25'}
]

progress_file = 'progress.json'
progress = load_progress(progress_file)

default_to = datetime.now()
default_from = default_to - timedelta(days=1)

for pool in pools:
    key = (pool['network'], pool['pool_address'])
    key_str = str(key)
    filename = f"{key[0]}_{key[1]}.csv"
    
    print(f"Checking pool: {key}")
    if key_str in progress and progress[key_str]['status'] == 'complete':
        print(f"Skipping completed: {key}")
        continue
    
    # Parse dates
    if 'from_date' in pool and 'to_date' in pool:
        from_dt = datetime.strptime(pool['from_date'], '%Y-%m-%d')
        to_dt = datetime.strptime(pool['to_date'], '%Y-%m-%d')
    else:
        from_dt = default_from
        to_dt = default_to
    
    from_ts = int(from_dt.timestamp())
    to_ts = int(to_dt.timestamp())
    
    if key_str in progress and progress[key_str]['status'] == 'partial':
        from_ts = max(from_ts, progress[key_str]['last_ts'])
        print(f"Resuming from last_ts: {datetime.fromtimestamp(from_ts)}")
    
    print(f"Fetching OHLCV for {key} from {datetime.fromtimestamp(from_ts)} to {datetime.fromtimestamp(to_ts)}")
    ohlcv = get_ohlcv(pool['network'], pool['pool_address'], from_ts, to_ts, filename)
    
    if ohlcv:
        save_ohlcv_to_csv(ohlcv, filename)
        save_progress(progress_file, key, 'complete')
    else:
        if os.path.exists(filename):
            df = pd.read_csv(filename)
            if not df.empty:
                last_ts = int(pd.to_datetime(df['timestamp']).min().timestamp())
                save_progress(progress_file, key, 'partial', last_ts)
                print(f"Partial progress saved for {key}. Restart from {datetime.fromtimestamp(last_ts)}")
        else:
            save_progress(progress_file, key, 'failed', to_ts)
            print(f"No data saved for {key}. Marked as failed.")

Checking pool: ('solana', 'EXQfMJKTnhHRv3xCzPttk9sZp6MKMdipeDnjKbGJq4fy')
Skipping completed: ('solana', 'EXQfMJKTnhHRv3xCzPttk9sZp6MKMdipeDnjKbGJq4fy')
Checking pool: ('solana', 'EwUU8oiKTomjekU5eszctx3kneKJnKm5BJpn5sTT4CUf')
Skipping completed: ('solana', 'EwUU8oiKTomjekU5eszctx3kneKJnKm5BJpn5sTT4CUf')
Checking pool: ('solana', 'BgdLpSBcEny1QZLMQGxW42az4iAb38ijwq5bxyHTs1mV')
Skipping completed: ('solana', 'BgdLpSBcEny1QZLMQGxW42az4iAb38ijwq5bxyHTs1mV')
Checking pool: ('solana', '7rQd8FhC1rimV3v9edCRZ6RNFsJN1puXM9UmjaURJRNj')
Skipping completed: ('solana', '7rQd8FhC1rimV3v9edCRZ6RNFsJN1puXM9UmjaURJRNj')
Checking pool: ('solana', 'e5h5bxlranyjfhezvr3r2j6kgdq3fnx8spkssgyqyya8')
Fetching OHLCV for ('solana', 'e5h5bxlranyjfhezvr3r2j6kgdq3fnx8spkssgyqyya8') from 2025-05-01 00:00:00 to 2025-08-25 00:00:00
Requesting OHLCV: https://api.geckoterminal.com/api/v2/networks/solana/pools/e5h5bxlranyjfhezvr3r2j6kgdq3fnx8spkssgyqyya8/ohlcv/minute with before_timestamp=2025-08-25 00:00:00
HTTP error 